# Tutorial 2.2: Deriving the EOM of the pendulum covered during the lecture
In this tutorial you will learn to derive the Equations of Motion of the pendulum as covered in the lecture. That is, a pendulum attached to the floor with a spring. We also consider an external horizontal force acting on the mass.

## Part 1: Kinematic equations
Using the following rotation matrix R($\phi$): https://en.wikipedia.org/wiki/Rotation_matrix
The zero angle points to the right for this matrix.

We first start by defining the variables. Assuming that x1, z1 and r1 do not change in time:

In [1]:
pip install sympy;

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sympy import *

var("t s x1 z1 r1")      
phi1 = Function("phi1")(t)

# Define the kinematic relations here

The velocities can then be obtained using:

In [5]:
# Compute/define the velocities here

## Part 2: Energy equations
### Kinetic energy:
The only mass in this system is the point mass located at P2 with mass $M$

In [6]:
var("M")
# Define the kinetic energy here (T)

M

This expression can be simplified to:

In [7]:
#T = simplify(T)
#T.evalf()

### Potential energy:
Assuming again that $l0$, the original length of the spring, does not change in time, then the elongation of the spring $dl$ is given by:

In [8]:
var("dx dz l0 x3 z3")
# Define the spring elongation here dl(l0,x2,z2,x3,z3)

(dx, dz, l0, x3, z3)

The work done by the spring between P2 and P3 with stiffness $k$ and gravity.

In [9]:
var("k g")
# Define the potential energy here (V)

(k, g)

### Work by external force
The work done by an external force working in the horizontal direction:

In [8]:
#Fx = Function("Fx")(t)
# Define your external work here (W)

(r1*cos(phi1(t)) + x1)*Fx(t)

## Step 3: Construct the Lagrangian

In [11]:
# Define your Lagrangian here (L)

## Step 4: Obtaining the EoM

In order to obtain the EoMs we have to take derivatives w.r.t. $\phi_1$ and its velocity. 

In [10]:
# Compute the EOM here

M*g*r1*cos(phi1(t)) + 1.0*M*r1**2*Derivative(phi1(t), (t, 2)) + 1.0*k*(-l0 + ((-r1*sin(phi1(t)) - z1 + z3)**2 + (-r1*cos(phi1(t)) - x1 + x3)**2)**0.5)*(-r1*(-r1*sin(phi1(t)) - z1 + z3)*cos(phi1(t)) + r1*(-r1*cos(phi1(t)) - x1 + x3)*sin(phi1(t)))/((-r1*sin(phi1(t)) - z1 + z3)**2 + (-r1*cos(phi1(t)) - x1 + x3)**2)**0.5 - r1*Fx(t)*sin(phi1(t))

Now we isolate it for the acceleration

In [11]:
# Isolate the acceleration

(-M*g*((r1*sin(phi1(t)) + z1 - z3)**2 + (r1*cos(phi1(t)) + x1 - x3)**2)**0.5*cos(phi1(t)) + k*l0*(-x1*sin(phi1(t)) + x3*sin(phi1(t)) + z1*cos(phi1(t)) - z3*cos(phi1(t))) + ((r1*sin(phi1(t)) + z1 - z3)**2 + (r1*cos(phi1(t)) + x1 - x3)**2)**0.5*(k*x1*sin(phi1(t)) - k*x3*sin(phi1(t)) - k*z1*cos(phi1(t)) + k*z3*cos(phi1(t)) + Fx(t)*sin(phi1(t))))/(M*r1*((r1*sin(phi1(t)) + z1 - z3)**2 + (r1*cos(phi1(t)) + x1 - x3)**2)**0.5)

The EOM of an actual pendulum can be recovered by removing the spring and the external force. Note that this returns a cos because $\phi (t)=0$ was set to the right, rather than downward which is more conventional for a pendulum.

In [12]:
# Setting k=0 you should recover the pendulum equation found in tutorial 2_1

## Bonus: Obtaining the EOMs for nDOF system

If your system contains multiple DOFs, you will have to take the derivative of $L$ towards each of these separately, thereby obtaining the EOM for each DOF. Let's say you obtained the following two EOMs (the EOMs are just random things I entered):

In [13]:
var("m c k J d q")
u = Function("u")(t)
w = Function("w")(t)

eom1 = m*cos(w)*diff(u, (t, 2)) + m*sin(u)*diff(w, (t, 2)) - c*diff(w, t) - c*diff(w, t)**2*u*ln(sqrt(t))**2*exp(u*w) - k*u
eom1.evalf()

-c*u(t)*exp(u(t)*w(t))*log(sqrt(t))**2*Derivative(w(t), t)**2 - c*Derivative(w(t), t) - k*u(t) + m*sin(u(t))*Derivative(w(t), (t, 2)) + m*cos(w(t))*Derivative(u(t), (t, 2))

In [14]:
eom2 = J*w**2*diff(u, (t, 2)) + J*diff(w, t)*diff(w, (t, 2)) - d*diff(w, t) + q*u
eom2.evalf()

J*w(t)**2*Derivative(u(t), (t, 2)) + J*Derivative(w(t), t)*Derivative(w(t), (t, 2)) - d*Derivative(w(t), t) + q*u(t)

You can collect both EOMs into matrix form using the following code:

In [23]:
# linear_eq_to_matrix only accepts symbols, therefore we have to substitute the second derivative functions with symbols a1 and a2
a1, a2 = symbols("a1 a2")
eom1 = eom1.evalf(subs={diff(u, (t, 2)): a1, diff(w, (t, 2)): a2})
eom2 = eom2.evalf(subs={diff(u, (t, 2)): a1, diff(w, (t, 2)): a2})
MTRX = linear_eq_to_matrix([eom1, eom2], [a1, a2])

Resulting in the following system of equations:

In [27]:
M = MTRX[0]
M.evalf()

Matrix([
[m*cos(w(t)),           m*sin(u(t))],
[  J*w(t)**2, J*Derivative(w(t), t)]])

And the following right-hand-side:

In [28]:
F = MTRX[1]
F.evalf()

Matrix([
[c*u(t)*exp(u(t)*w(t))*log(sqrt(t))**2*Derivative(w(t), t)**2 + c*Derivative(w(t), t) + k*u(t)],
[                                                               d*Derivative(w(t), t) - q*u(t)]])